<a href="https://colab.research.google.com/github/robingenz/htwg-machine-learning-exercises/blob/main/exercises/01_Explorative_Analyse_und_PCA/01-explorative-analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eigengesichter

## Setup

### Einbinden von Paketen

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import math
%matplotlib inline

np.__version__, pd.__version__, sklearn.__version__

('1.23.4', '1.5.0', '1.1.3')

In [19]:
%load_ext version_information
%version_information

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 3.10.8 64bit [GCC 10.2.1 20210110]
IPython 8.6.0
OS Linux 5.10.104 linuxkit aarch64 with glibc2.31
Sun Nov 13 19:28:03 2022 UTC

## Aufgaben

### Aufgabe a)

In [27]:
import os
from urllib.request import urlretrieve
import tarfile

tgz_file_path = 'data/lfw-funneled.tgz'
data_dir_path = 'data/lfw_funneled'

# Herunterladen des Datensatzes
if not os.path.isfile(tgz_file_path):
    print("Downloading...")
    urlretrieve('http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz', filename = tgz_file_path)
    print("Download finished.")

dateDownloaded = !date #Calling Linux
print(dateDownloaded)

# Entpacken des Datensatzes
if not os.path.isdir(data_dir_path):
    print("Extracting...")
    tar = tarfile.open(tgz_file_path, 'r:gz')
    tar.extractall(path='data/')
    tar.close()
    print("Extract finished.")

['Sun Nov 13 19:36:55 UTC 2022']


Finden von Personen mit **70 oder mehr** Bildern:

In [39]:
person_dir_paths = []
for subdir, _, files in os.walk(data_dir_path):
    if len(files) >= 70:
        person_dir_paths.append(subdir)

print('Found', len(person_dir_paths), 'persons with ≥ 70 images.')

Found 7 persons with ≥ 70 images.


### Aufgabe b)

Datensatz in Trainings- und Testbilder aufteilen.

In [43]:
from os import listdir

test_image_paths = []
training_image_paths = []

for person_dir_path in person_dir_paths:
    files = sorted(listdir(person_dir_path))
    test_image_paths.append(f'{person_dir_path}/{files[0]}')
    for i in range(1, len(files)):
        training_image_paths.append(f'{person_dir_path}/{files[i]}')

print('Splitted into', len(test_image_paths), 'test images and', len(training_image_paths), 'training images.')

Splitted into 7 test images and 1281 training images.
